In [141]:
from openai import OpenAI
import pandas as pd
import json
import pickle
from tqdm.auto import tqdm
import requests
from sklearn.model_selection import GroupKFold
import numpy as np
import glob
import torch
import pickle
import time
import random

In [142]:
path_prefix = "../data"
train = pd.read_csv(f"{path_prefix}/train.csv")
test = pd.read_csv(f"{path_prefix}/test.csv")
sample_submission = pd.read_csv(f"{path_prefix}/sample_submission.csv")
misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")
misconception_mapping_dict = {}
for _,row in misconception_mapping.iterrows():
    misconception_mapping_dict[row['MisconceptionId']] = row['MisconceptionName']

In [143]:
datas = []
for path in tqdm(glob.glob("./save_data/cv1/*.pkl")):
    with open(path,'rb') as f:
        data = pickle.load(f)
    datas.append((data,path))

In [144]:
df = []
for d,path in datas:
    mis_id = int(path.split("/")[-1].split("_",1)[0])
    mis_text = misconception_mapping_dict[mis_id]
    text = d[-1][0].strip('\n').strip()
    text_list = text.split("#$%")
    for text in text_list:
        try:
            subject_name = text.split("SubjectName:",1)[1].split("ConstructName:",1)[0].strip("\n").strip()
            construct_name = text.split("ConstructName:",1)[1].split("Question:",1)[0].strip("\n").strip()
            question = text.split("Question:",1)[1].split("CandidateAnswer:",1)[0].strip("\n").strip()
            correct_answer = text.split("CorrectAnswer:",1)[1].split("MisconceptionReason:",1)[0].strip("\n").strip()
            candidate_answer = text.split("Answer:",1)[1].split("SelectedAnswer:",1)[0].strip("\n").strip()
            selected_answer = text.split("SelectedAnswer:",1)[1].split("CorrectAnswer:",1)[0].strip("\n").strip()
            df.append({
                "SubjectName": subject_name,
                "ConstructName": construct_name,
                "Question": question,
                "CandidateAnswer": candidate_answer,
                "SelectedAnswer": selected_answer,
                "CorrectAnswer": correct_answer,
                "mis_id":mis_id
            })
        except:
            continue
df = pd.DataFrame(df)

In [145]:
df.to_parquet('save_data/cv1.parquet',index=False)

In [146]:
df.shape

(7691, 7)